In [12]:
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd
import sys

# setting path
sys.path.append('../app')

from src.scripts.similarity import get_similar_works
models_path = "../app/models"
embeddings_path = "../app/embeddings"

# Récupération du modèle non fine-tuné
model_1 = SentenceTransformer(f"{models_path}/sentence_similarity_model")

model_2 = SentenceTransformer(f"{models_path}/sentence_similarity_model_FT")

dataset = pd.read_csv("../Data/movie_synopsis.csv")

with open(f"{embeddings_path}/embeddings_corpus_movie", "rb") as f:
    embeddings = pickle.load(f)

In [13]:
def eval(model):
    value = 0
    for _, title, syn in embeddings:
        best = get_similar_works(user_input= syn, oeuvres=embeddings, model=model_1)[0]['title']
        if best == title:
            value += 1
        else:
            print(f"Confusion entre {title}, pris pour {best}")
    return value/len(embeddings)


In [14]:
eval(model_1)

KeyboardInterrupt: 

In [ ]:
eval(model_2)